süper — eğitim çıktıların gayet mantıklı 👌 şimdi iki şeyi yapalım:

1. Kodu parça parça, **“neden yaptık?”** diye açıklayayım (özellikle *Bernoulli–Bernoulli RBM* kısmı).
2. “**Kullanıcıya film önerisi nasıl alırım?**” için net örnek sorgu kodu vereyim (başlıklarıyla beraber).

---

# 1) Kodun Mantığı — Adım Adım

## (a) Veri → Kullanıcı×Film matrisi → Binarizasyon

* MovieLens dosyalarını `user, item, rating` olarak okuyoruz; sonra her kullanıcı için film sayısı uzunluğunda bir vektör oluşturuyoruz.
* **Binarize etme:**

  * 0 → **bilinmiyor** (kullanıcı o filme hiç puan vermemiş) → `-1` ile işaretliyoruz (maskede “eğitme”).
  * 1–2 → **beğenmedi** → `0`
  * 3–5 → **beğendi** → `1`
* Neden? RBM görünür birimleri **Bernoulli (0/1)** kabul ediyor; yani “bu kullanıcı bu filmi sever mi?” sorusuna ikili bir cevap.

## (b) RBM: Bernoulli–Bernoulli nedir?

* **Görünür katman** $v \in {0,1}^{N_\text{items}}$: her film için 0/1 (sevmez/sever).
* **Gizli katman** $h \in {0,1}^{N_\text{hidden}}$: filmler arasındaki **latent faktörleri** temsil eder (ör. “bilimkurgu sevgisi”, “romantik komedi teması” gibi).
* Enerji fonksiyonu:

  $$
  E(v,h) = - v^\top W^\top h - b^\top v - a^\top h
  $$
* Koşullu olasılıklar (bağımsız Bernoulli):

  * $p(h=1 \mid v) = \sigma(v W^\top + a)$
  * $p(v=1 \mid h) = \sigma(h W + b)$
* Dolayısıyla `sample_h(v)` ve `sample_v(h)` fonksiyonlarında **sigmoid** ile olasılık hesaplanıp gerekirse Bernoulli örnekleniyor.

## (c) Contrastive Divergence (CD-k)

* **Amaç:** Gerçek verinin ortak aktivasyonunu ⟨v·hᵀ⟩ ile, modelin kendi ürettiği örneklerin ortak aktivasyonunu ⟨v'·h'ᵀ⟩ **yaklaştırmak**.
* Adımlar:

  1. Gerçek veri $v_0$ → $h_0 \sim p(h\mid v_0)$
  2. $k$ adım Gibbs zinciri: $h\to v \to h \to \dots$
  3. Güncelleme:

     $$
     \Delta W \propto \langle v_0 h_0^\top\rangle - \langle v_k h_k^\top\rangle
     $$
* Biz `update` içinde bunu **mini-batch ortalama**, **öğrenme oranı**, **momentum** ve **weight decay** ile yapıyoruz:

  * **Momentum**: titreşimi azaltır, daha kararlı güncelleme.
  * **Weight decay (L2)**: aşırı ezberlemeyi önler.

## (d) Maskeli eğitim (bilinmeyenleri taşıma)

* Veri matriksinde **bilinmeyen** girişleri `-1` ile işaretledik.
* CD içinde görünür örneği güncellerken **yalnızca bilinen (0 veya 1)** pozisyonlarda güncelliyoruz:

  ```python
  mask = (v0 >= 0).float()
  vk = v0 * (1 - mask) + vk_sample * mask
  ```

  Böylece “kullanıcı hakkında bilmediğimiz filmler” **öğretim sinyali** üretmiyor (gürültü katmıyor).

## (e) Rekonstrüksiyon hatası

* Eğitim ve testte raporladığın “**reconstruction loss**”, **bilinen** girdilerde `|v0 - v_recon|`’un ortalaması.
* Bu, RBM’nin “gördüğünü geri yazma kapasitesini” ölçer; **öneri kalitesinin** birebir karşılığı değildir. O yüzden ayrıca **Precision\@K / Recall\@K** veriyoruz (sıralama kalitesi).

## (f) Precision\@10 / Recall\@10

* Her kullanıcı için modelin verdiği **olasılık skorlarına** göre **en iyi 10 film** seçiyoruz.
* **Precision\@10:** Bu 10’luk listenin kaç tanesi testte gerçekten pozitif (beğendi)?
* **Recall\@10:** Testteki tüm pozitiflerinden kaçını ilk 10’da yakaladın?

---

# 2) “Kullanıcıya film önerisi” — nasıl sorgularım?

Aşağıdaki kod, **tek bir kullanıcı** için **top-K öneriyi** verir.

* Eğitimde gördüğü pozitifleri listeden **hariç** tutuyoruz (yeni öneri istiyoruz).
* Sonucu **film başlıklarıyla** döndürüyoruz.

```python
import numpy as np
import pandas as pd
import torch

# 1) Film başlıklarını oku (ml-1m için)
# movies.dat biçimi: MovieID::Title::Genres
movies = pd.read_csv("ml-1m/movies.dat", sep="::", header=None, engine="python", encoding="latin-1")
movies.columns = ["movie_id", "title", "genres"]

# 2) Yardımcı: kullanıcı vektörünü al (binarize edilmiş train_bin'den)
#    train_bin: (nb_users, nb_items) numpy float32; 1=pozitif, 0=negatif, -1=bilinmiyor
def get_user_vector(user_id_1_based, train_bin):
    # RBM kodunda kullanıcı indeksleri 0-bazlı; MovieLens ise 1-bazlı idi
    return train_bin[user_id_1_based - 1].copy()

# 3) Modelden olasılık skorlarını üret (p(v=1|h))
@torch.no_grad()
def rbm_predict_proba_for_user(rbm, user_vec_np):
    # user_vec_np: shape (nb_items,), değerler {-1,0,1}
    v = torch.tensor(user_vec_np, dtype=torch.float32, device=rbm.W.device).unsqueeze(0)  # (1, nv)
    ph, _ = rbm.sample_h(v)
    pv, _ = rbm.sample_v(ph)   # (1, nv)
    return pv.squeeze(0).cpu().numpy()  # (nv,)

# 4) Tek kullanıcı için top-K öneri
def recommend_topk_for_user(user_id_1b, K, rbm, train_bin, movies_df):
    user_vec = get_user_vector(user_id_1b, train_bin)
    proba = rbm_predict_proba_for_user(rbm, user_vec)

    # Zaten kullanıcının pozitif verdiği filmleri öneriden çıkar (yeni öneri)
    already_pos = np.where(user_vec == 1.0)[0]
    proba[already_pos] = -1e9

    # (Opsiyonel) açıkça 0 verdiği (beğenmediği) filmleri de çıkarabilirsin:
    # already_neg = np.where(user_vec == 0.0)[0]
    # proba[already_neg] = -1e9

    topk_idx = np.argsort(proba)[-K:][::-1]
    # MovieID’ler 1-bazlı olduğu için +1
    rec_movie_ids = (topk_idx + 1)

    # Başlıkları eşleştir
    rec = movies_df[movies_df["movie_id"].isin(rec_movie_ids)][["movie_id","title","genres"]]
    # Sıralamayı proba’ya göre yapalım
    order = {mid: proba[mid-1] for mid in rec_movie_ids}
    rec = rec.sort_values(by="movie_id", key=lambda s: s.map(order), ascending=False)
    rec["score"] = rec["movie_id"].map(order)
    return rec.reset_index(drop=True)

# --- Örnek kullanım:
user_id = 42
K = 10
recs = recommend_topk_for_user(user_id, K, rbm, train_bin, movies)
print(recs)
```

> Eğer ml-100k kullanıyorsan film meta dosyası `u.item` (ayrı format). Onu da `sep="|"` ile okuyup `movie_id → title` eşleştirmesi yapabilirsin. Temel mantık aynı.

---

## Sık sorulan ikililer

**Bernoulli–Bernoulli** ne demek tekrar?
→ Hem görünür birimler (filmler) hem de gizli birimler (faktörler) **ikili (0/1)** olarak modelleniyor; koşullu dağılımlar **sigmoid**.

**Reconstruction loss iyi gidiyor ama Precision\@10 düşük kalırsa?**
→ Normal; rekonstrüksiyon “yeniden yazma” ölçüsü. **Öneri** için sıralama metriklerini (Precision/Recall\@K, MAP, NDCG) optimize eden yöntemler (BPR, WARP, NCF) çoğu zaman daha yüksek olur. RBM burada **temel davranış öğrenmeyi** gösterir.

**Skoru nasıl yükseltirim?**

* **nh** (gizli birim sayısı) ↑: 64/128/256 dene.
* **CD k**: 1→5→10 karşılaştır.
* **lr, momentum, weight\_decay** küçük grid-search.
* **Epoch** ↑ (erken durdurma ile).
* **Veri ön işleme:** Kullanıcı/film başına min etkileşim eşiği (çok seyrek kullanıcı/film filtrele).
* **Hybrid**: İçerik özellikleri ekle (film türü, yıla göre bias vb.) → “görünür”ü çok-logit (softmax) olarak modellemek gelişmiş konu; sıradaki adım olabilir.

**Cold-start (yeni kullanıcı/film)?**
→ RBM gibi saf etkileşim temelli yöntemlerde zordur. İçerik tabanlı özellikler (film genre, yıl, oyuncular) veya **meta-öğrenme** gerekir.

---

## Sonuçları nasıl yorumlamalı?

* **Train recon loss ↓**: model veriyi daha iyi yeniden kuruyor; yakınsama var.
* **Test recon loss ≈ 0.27**: makul.
* **Precision\@10 ≈ 0.10, Recall\@10 ≈ 0.027** (ml-100k için başlangıç seviyesi normaldir).

  * P\@10 = %10 → ilk 10 öneride ort. 1 tanesi gerçekten pozitif.
  * R\@10 = %2.7 → kullanıcının tüm pozitiflerinden küçük bir kısmı yakalanıyor.
* Bu metrikleri yukarıdaki **tuning** adımlarıyla yavaş yavaş yükseltebilirsin.

---

hazırsan: istersen **ml-100k için film başlığı eşleştirme** (u.item formatı) örneğini de ayrıca atayım, ya da **Goodbooks-10k** gibi başka bir veri setine aynı RBM pipeline’ını uyarlayalım.


Kesinlikle hayır—Boltzmann makineleri (özellikle **RBM**) sadece film önerileri için değildir. “**Kullanıcı–öğe etkileşimi**” olan **her** alanda (alışveriş, müzik, kitap, restoran, oyun, haber vb.) **ikili tercih / implicit feedback** üretip öneri yapılabilir. Ayrıca RBM’ler generatif/densite modellemede (ör. binarize MNIST), anomali tespitinde ve özellik çıkarımında da kullanılır.

Aşağıya önce veri seti önerilerini, ardından elindeki MovieLens kodunun **daha sağlam** bir sürümünü (öğrenme oranı, momentum, v.b. ile) ve değerlendirmenin nasıl yapılacağını bıraktım.

---

# ✅ RBM ile çalışabileceğin veri setleri

## Öneri / kullanıcı–öğe etkileşimi

* **MovieLens** (ml-100k, ml-1m, ml-20m): Klasik benchmark; filmler için puanlar.
* **Goodbooks-10k**: Kitap puanları – kitap önerisi.
* **Jester**: Fıkra şakalarına verilen dereceler – RBM ile iyi çalışır.
* **Last.fm / Million Song Dataset**: Müzik oynatma ve etkileşim – dinleme tabanlı öneri.
* **Amazon Reviews** (çeşitli kategoriler): Ürün–kullanıcı puan/yorumları – “beğendim/beğenmedim”e binarize edilebilir.
* **Yelp**: Restoran/işletme puanları – yemek/yer önerileri.
* **Steam / Games**: Oyun sahiplikleri/incelemeleri – oyun önerisi.
* **Book-Crossing**: Kitap rating’leri – kitap önerisi.

> Ortak fikir: **kullanıcı × öğe** matrisini **binarize** et (örn. rating ≥ 3 → 1, yoksa 0; “bilinmiyor”ları -1 ile maskele) ve RBM’yi **Bernoulli görünür** birimlerle kur.

## Öneri dışı (RBM’nin başka alanları)

* **Binarize MNIST** (0/1 piksel): Generatif modelleme & özellik çıkarım.
* **Tıklama/görüntüleme günlükleri (clickstream)**: İçerik/ürün önerisi.
* **Kurumsal erişim logları**: RBM ile temel yoğunluk modeli → **anomali skoru** (alışılmadık oturumlar).

---

# 🧠 RBM kısaca (neden işe yarıyor?)

* **İkili görünür birimler** (v) ve **ikili gizli birimler** (h) arasında iki katmanlı **enerji tabanlı** model.
* **Contrastive Divergence (CD-k)** ile eğitim:

  1. Veri görünür v₀ → h \~ p(h|v₀)
  2. h → vₖ \~ p(v|h) (k adım Gibbs)
  3. Ağırlık güncelle: ΔW ∝ ⟨v₀h₀ᵀ⟩ − ⟨vₖhₖᵀ⟩ (öğrenme oranı, momentum, reg. ile)

Senin eğitim kodun “çekirdek” olarak doğru ama **öğrenme oranı, momentum, regülarizasyon** yok; ayrıca **değerlendirme** olarak sadece “rekonstrüksiyon hatası” bakıyor. Aşağıda iyileştirilmiş, **PyTorch** ile çalışan bir RBM iskeleti var.

---

# 🧩 MovieLens (ml-100k) için iyileştirilmiş RBM (PyTorch)

> Not: Bu sürüm **öğrenme oranı, momentum, weight decay** içerir; **CD-k** uygular; **maskeli kayıp** kullanır. Ayrıca **Precision\@K / Recall\@K** örneği var ki öneri için daha anlamlıdır.

```python
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 1) Veri yükleme (ml-100k split)
train = pd.read_csv("ml-100k/u1.base", delimiter="\t", header=None, names=["user","item","rating","ts"])
test  = pd.read_csv("ml-100k/u1.test",  delimiter="\t", header=None, names=["user","item","rating","ts"])

nb_users  = max(train.user.max(), test.user.max())
nb_items  = max(train.item.max(), test.item.max())

def to_user_item_matrix(df, nb_users, nb_items):
    X = np.zeros((nb_users, nb_items), dtype=np.float32)
    for u, it, r, _ in df.values:
        X[u-1, it-1] = r
    return X

train_mat = to_user_item_matrix(train, nb_users, nb_items)
test_mat  = to_user_item_matrix(test,  nb_users, nb_items)

# 2) Binarize + mask
#  rating == 0 -> bilinmiyor (-1); 1-2 -> 0; 3-5 -> 1
def binarize(x):
    y = x.copy()
    y[y == 0]  = -1.0
    y[(y==1) | (y==2)] = 0.0
    y[y >= 3]  = 1.0
    return y

train_bin = binarize(train_mat)
test_bin  = binarize(test_mat)

train_t = torch.tensor(train_bin, device=device)
test_t  = torch.tensor(test_bin,  device=device)

# 3) RBM tanımı (Bernoulli-Bernoulli)
class RBM(nn.Module):
    def __init__(self, nv, nh):
        super().__init__()
        # W: nh x nv (PyTorch param değil; manuel güncelleyeceğiz)
        self.W = torch.randn(nh, nv, device=device) * 0.01
        self.hbias = torch.zeros(1, nh, device=device)
        self.vbias = torch.zeros(1, nv, device=device)

    def sample_h(self, v):
        # p(h=1|v) = sigmoid(v W^T + hbias)
        prob = torch.sigmoid(v @ self.W.t() + self.hbias)
        return prob, torch.bernoulli(prob)

    def sample_v(self, h):
        # p(v=1|h) = sigmoid(h W + vbias)
        prob = torch.sigmoid(h @ self.W + self.vbias)
        return prob, torch.bernoulli(prob)

    @torch.no_grad()
    def update(self, v0, vk, ph0, phk, lr=0.01, momentum=0.5, weight_decay=1e-4,
               vW=None, vh=None, vv=None):
        # momentum buffer'ları
        if vW is None:
            vW = torch.zeros_like(self.W)
            vh = torch.zeros_like(self.hbias)
            vv = torch.zeros_like(self.vbias)

        dW = (ph0.t() @ v0 - phk.t() @ vk) / v0.size(0)  # ortalama
        db = torch.sum(v0 - vk, dim=0, keepdim=True) / v0.size(0)
        da = torch.sum(ph0 - phk, dim=0, keepdim=True) / v0.size(0)

        # weight decay (L2 benzeri)
        dW -= weight_decay * self.W

        # momentum güncellemesi
        vW = momentum * vW + lr * dW
        vv = momentum * vv + lr * db
        vh = momentum * vh + lr * da

        self.W     += vW
        self.vbias += vv
        self.hbias += vh
        return vW, vh, vv

nv, nh = nb_items, 128
rbm = RBM(nv, nh)

# 4) Mini-batch eğitim (CD-k)
batch_size = 128
epochs = 15
k = 5          # CD-k adımı
lr = 0.05
momentum = 0.5
weight_decay = 1e-4

dataset = TensorDataset(train_t)
loader = DataLoader(dataset, batch_size=batch_size, shuffle=True, drop_last=True)

vW = vh = vv = None
for epoch in range(1, epochs+1):
    running = 0.0
    cnt = 0
    for (v0,) in loader:
        # v0: (B, nv)
        vk = v0.clone()
        # mask: bilinmeyenler (-1) bu pozisyonları koru
        mask = (v0 >= 0).float()

        ph0, _ = rbm.sample_h(v0)
        for _ in range(k):
            _, hk = rbm.sample_h(vk)
            pvk, vk_sample = rbm.sample_v(hk)
            # bilinmeyen pozisyonları (mask==0) v0’dan taşımayız; sadece bilinenleri güncelliyoruz
            vk = v0 * (1 - mask) + vk_sample * mask

        phk, _ = rbm.sample_h(vk)

        vW, vh, vv = rbm.update(v0, vk, ph0, phk, lr, momentum, weight_decay, vW, vh, vv)

        # rekonstrüksiyon hatası: sadece bilinenlerde
        loss = torch.mean(torch.abs(v0[mask.bool()] - vk[mask.bool()]))
        running += loss.item()
        cnt += 1
    print(f"Epoch {epoch:02d} | train recon loss: {running/cnt:.4f}")

# 5) Test rekonstrüksiyon hatası
with torch.no_grad():
    v = train_t  # görünür başlangıç olarak train’i kullan (standart yaklaşım)
    ph, _ = rbm.sample_h(v)
    pv, _ = rbm.sample_v(ph)
    mask_te = (test_t >= 0).float()
    test_loss = torch.mean(torch.abs(test_t[mask_te.bool()] - pv[mask_te.bool()]))
    print("Test recon loss:", float(test_loss))
```

## 🎯 Öneri (top-N) değerlendirmesi — Precision\@K / Recall\@K

Rekonstrüksiyon hatası “öneri kalitesini” tam yansıtmaz. Aşağıda kullanıcı bazında **pozitif test öğelerini** hedef alıp **Precision\@10 / Recall\@10** hesaplayan basit bir örnek var (negatif örnekleri basitçe “0” sayıyoruz; daha adil bir kıyas için “negative sampling” ve “unrated filtreleme” eklenebilir).

```python
def topk_metrics(train_bin, test_bin, proba, K=10):
    # train'de zaten 1 olanları öneriden hariç tutmak istersen maskelersin.
    P = []
    R = []
    for u in range(train_bin.shape[0]):
        # testteki pozitifler
        pos = np.where(test_bin[u] == 1.0)[0]
        if len(pos) == 0:
            continue
        scores = proba[u].copy()

        # (Opsiyonel) zaten train'de 1 olanları öneriden çıkar
        already_pos = np.where(train_bin[u] == 1.0)[0]
        scores[already_pos] = -1e9

        topk = np.argsort(scores)[-K:][::-1]
        hit = len(set(topk) & set(pos))
        P.append(hit / K)
        R.append(hit / len(pos))
    return float(np.mean(P)), float(np.mean(R))

with torch.no_grad():
    ph, _ = rbm.sample_h(train_t)
    pv, _ = rbm.sample_v(ph)      # öneri olasılıkları ~ p(v=1|h)
    proba = pv.detach().cpu().numpy()
    P10, R10 = topk_metrics(train_bin, test_bin, proba, K=10)
    print(f"Precision@10={P10:.4f} | Recall@10={R10:.4f}")
```

> Daha adil değerlendirme için: “bilinmeyenleri” negatif sayma; her kullanıcı için test pozitiflerine karşı **rastgele negatif örnekle** ve **AUC / MAP / NDCG** gibi metrikler kullan.

---

## SSS (kısa notlar)

* **RBM sadece MovieLens için mi?**
  Hayır. Her **user–item** matrisinde (müzik, kitap, ürün, restoran, oyun…) **binarize** edip RBM ile **implicit öneri** yapılabilir.

* **Neden rating’leri binarize ediyoruz?**
  Bu RBM **Bernoulli görünür** birimler varsayıyor. Çok seviyeli rating için **Gaussian–Bernoulli RBM** veya **softmax görünür** (her item için puan sınıfları) kullanılabilir; ama pratikte “beğendi/beğenmedi (implicit)” iyi çalışır.

* **CD-k kaç olmalı?**
  1–10 arası yaygın. k↑ → daha iyi yakınsama ama daha yavaş. Genelde **CD-1..CD-5** yeterli.

* **Rekonstrüksiyon kaybı neden yeterli değil?**
  Önerinin kalitesi **sıralama** ile ölçülür (Precision\@K, Recall\@K, MAP, NDCG). O yüzden ekledik.

* **Modern alternatifler?**
  RBM eğitici bir temel. Pratikte **Matrix Factorization (BPR/ALS)**, **Neural CF**, **AutoEncoder (VAE/DAE)**, **LightFM** ve **implicit** kütüphanesi gibi yöntemler sık kullanılıyor.

---

İstersen yukarıdaki kodu **kendi veri setin** için uyarlayalım (ör. Goodbooks-10k veya Amazon). “Veriyi nasıl okuyalım, nasıl binarize edelim, split nasıl olsun?” diye beraber net bir pipeline çıkarırız.


süper — eğitim çıktıların gayet mantıklı 👌 şimdi iki şeyi yapalım:

1. Kodu parça parça, **“neden yaptık?”** diye açıklayayım (özellikle *Bernoulli–Bernoulli RBM* kısmı).
2. “**Kullanıcıya film önerisi nasıl alırım?**” için net örnek sorgu kodu vereyim (başlıklarıyla beraber).

---

# 1) Kodun Mantığı — Adım Adım

## (a) Veri → Kullanıcı×Film matrisi → Binarizasyon

* MovieLens dosyalarını `user, item, rating` olarak okuyoruz; sonra her kullanıcı için film sayısı uzunluğunda bir vektör oluşturuyoruz.
* **Binarize etme:**

  * 0 → **bilinmiyor** (kullanıcı o filme hiç puan vermemiş) → `-1` ile işaretliyoruz (maskede “eğitme”).
  * 1–2 → **beğenmedi** → `0`
  * 3–5 → **beğendi** → `1`
* Neden? RBM görünür birimleri **Bernoulli (0/1)** kabul ediyor; yani “bu kullanıcı bu filmi sever mi?” sorusuna ikili bir cevap.

## (b) RBM: Bernoulli–Bernoulli nedir?

* **Görünür katman** $v \in {0,1}^{N_\text{items}}$: her film için 0/1 (sevmez/sever).
* **Gizli katman** $h \in {0,1}^{N_\text{hidden}}$: filmler arasındaki **latent faktörleri** temsil eder (ör. “bilimkurgu sevgisi”, “romantik komedi teması” gibi).
* Enerji fonksiyonu:

  $$
  E(v,h) = - v^\top W^\top h - b^\top v - a^\top h
  $$
* Koşullu olasılıklar (bağımsız Bernoulli):

  * $p(h=1 \mid v) = \sigma(v W^\top + a)$
  * $p(v=1 \mid h) = \sigma(h W + b)$
* Dolayısıyla `sample_h(v)` ve `sample_v(h)` fonksiyonlarında **sigmoid** ile olasılık hesaplanıp gerekirse Bernoulli örnekleniyor.

## (c) Contrastive Divergence (CD-k)

* **Amaç:** Gerçek verinin ortak aktivasyonunu ⟨v·hᵀ⟩ ile, modelin kendi ürettiği örneklerin ortak aktivasyonunu ⟨v'·h'ᵀ⟩ **yaklaştırmak**.
* Adımlar:

  1. Gerçek veri $v_0$ → $h_0 \sim p(h\mid v_0)$
  2. $k$ adım Gibbs zinciri: $h\to v \to h \to \dots$
  3. Güncelleme:

     $$
     \Delta W \propto \langle v_0 h_0^\top\rangle - \langle v_k h_k^\top\rangle
     $$
* Biz `update` içinde bunu **mini-batch ortalama**, **öğrenme oranı**, **momentum** ve **weight decay** ile yapıyoruz:

  * **Momentum**: titreşimi azaltır, daha kararlı güncelleme.
  * **Weight decay (L2)**: aşırı ezberlemeyi önler.

## (d) Maskeli eğitim (bilinmeyenleri taşıma)

* Veri matriksinde **bilinmeyen** girişleri `-1` ile işaretledik.
* CD içinde görünür örneği güncellerken **yalnızca bilinen (0 veya 1)** pozisyonlarda güncelliyoruz:

  ```python
  mask = (v0 >= 0).float()
  vk = v0 * (1 - mask) + vk_sample * mask
  ```

  Böylece “kullanıcı hakkında bilmediğimiz filmler” **öğretim sinyali** üretmiyor (gürültü katmıyor).

## (e) Rekonstrüksiyon hatası

* Eğitim ve testte raporladığın “**reconstruction loss**”, **bilinen** girdilerde `|v0 - v_recon|`’un ortalaması.
* Bu, RBM’nin “gördüğünü geri yazma kapasitesini” ölçer; **öneri kalitesinin** birebir karşılığı değildir. O yüzden ayrıca **Precision\@K / Recall\@K** veriyoruz (sıralama kalitesi).

## (f) Precision\@10 / Recall\@10

* Her kullanıcı için modelin verdiği **olasılık skorlarına** göre **en iyi 10 film** seçiyoruz.
* **Precision\@10:** Bu 10’luk listenin kaç tanesi testte gerçekten pozitif (beğendi)?
* **Recall\@10:** Testteki tüm pozitiflerinden kaçını ilk 10’da yakaladın?

---

# 2) “Kullanıcıya film önerisi” — nasıl sorgularım?

Aşağıdaki kod, **tek bir kullanıcı** için **top-K öneriyi** verir.

* Eğitimde gördüğü pozitifleri listeden **hariç** tutuyoruz (yeni öneri istiyoruz).
* Sonucu **film başlıklarıyla** döndürüyoruz.

```python
import numpy as np
import pandas as pd
import torch

# 1) Film başlıklarını oku (ml-1m için)
# movies.dat biçimi: MovieID::Title::Genres
movies = pd.read_csv("ml-1m/movies.dat", sep="::", header=None, engine="python", encoding="latin-1")
movies.columns = ["movie_id", "title", "genres"]

# 2) Yardımcı: kullanıcı vektörünü al (binarize edilmiş train_bin'den)
#    train_bin: (nb_users, nb_items) numpy float32; 1=pozitif, 0=negatif, -1=bilinmiyor
def get_user_vector(user_id_1_based, train_bin):
    # RBM kodunda kullanıcı indeksleri 0-bazlı; MovieLens ise 1-bazlı idi
    return train_bin[user_id_1_based - 1].copy()

# 3) Modelden olasılık skorlarını üret (p(v=1|h))
@torch.no_grad()
def rbm_predict_proba_for_user(rbm, user_vec_np):
    # user_vec_np: shape (nb_items,), değerler {-1,0,1}
    v = torch.tensor(user_vec_np, dtype=torch.float32, device=rbm.W.device).unsqueeze(0)  # (1, nv)
    ph, _ = rbm.sample_h(v)
    pv, _ = rbm.sample_v(ph)   # (1, nv)
    return pv.squeeze(0).cpu().numpy()  # (nv,)

# 4) Tek kullanıcı için top-K öneri
def recommend_topk_for_user(user_id_1b, K, rbm, train_bin, movies_df):
    user_vec = get_user_vector(user_id_1b, train_bin)
    proba = rbm_predict_proba_for_user(rbm, user_vec)

    # Zaten kullanıcının pozitif verdiği filmleri öneriden çıkar (yeni öneri)
    already_pos = np.where(user_vec == 1.0)[0]
    proba[already_pos] = -1e9

    # (Opsiyonel) açıkça 0 verdiği (beğenmediği) filmleri de çıkarabilirsin:
    # already_neg = np.where(user_vec == 0.0)[0]
    # proba[already_neg] = -1e9

    topk_idx = np.argsort(proba)[-K:][::-1]
    # MovieID’ler 1-bazlı olduğu için +1
    rec_movie_ids = (topk_idx + 1)

    # Başlıkları eşleştir
    rec = movies_df[movies_df["movie_id"].isin(rec_movie_ids)][["movie_id","title","genres"]]
    # Sıralamayı proba’ya göre yapalım
    order = {mid: proba[mid-1] for mid in rec_movie_ids}
    rec = rec.sort_values(by="movie_id", key=lambda s: s.map(order), ascending=False)
    rec["score"] = rec["movie_id"].map(order)
    return rec.reset_index(drop=True)

# --- Örnek kullanım:
user_id = 42
K = 10
recs = recommend_topk_for_user(user_id, K, rbm, train_bin, movies)
print(recs)
```

> Eğer ml-100k kullanıyorsan film meta dosyası `u.item` (ayrı format). Onu da `sep="|"` ile okuyup `movie_id → title` eşleştirmesi yapabilirsin. Temel mantık aynı.

---

## Sık sorulan ikililer

**Bernoulli–Bernoulli** ne demek tekrar?
→ Hem görünür birimler (filmler) hem de gizli birimler (faktörler) **ikili (0/1)** olarak modelleniyor; koşullu dağılımlar **sigmoid**.

**Reconstruction loss iyi gidiyor ama Precision\@10 düşük kalırsa?**
→ Normal; rekonstrüksiyon “yeniden yazma” ölçüsü. **Öneri** için sıralama metriklerini (Precision/Recall\@K, MAP, NDCG) optimize eden yöntemler (BPR, WARP, NCF) çoğu zaman daha yüksek olur. RBM burada **temel davranış öğrenmeyi** gösterir.

**Skoru nasıl yükseltirim?**

* **nh** (gizli birim sayısı) ↑: 64/128/256 dene.
* **CD k**: 1→5→10 karşılaştır.
* **lr, momentum, weight\_decay** küçük grid-search.
* **Epoch** ↑ (erken durdurma ile).
* **Veri ön işleme:** Kullanıcı/film başına min etkileşim eşiği (çok seyrek kullanıcı/film filtrele).
* **Hybrid**: İçerik özellikleri ekle (film türü, yıla göre bias vb.) → “görünür”ü çok-logit (softmax) olarak modellemek gelişmiş konu; sıradaki adım olabilir.

**Cold-start (yeni kullanıcı/film)?**
→ RBM gibi saf etkileşim temelli yöntemlerde zordur. İçerik tabanlı özellikler (film genre, yıl, oyuncular) veya **meta-öğrenme** gerekir.

---

## Sonuçları nasıl yorumlamalı?

* **Train recon loss ↓**: model veriyi daha iyi yeniden kuruyor; yakınsama var.
* **Test recon loss ≈ 0.27**: makul.
* **Precision\@10 ≈ 0.10, Recall\@10 ≈ 0.027** (ml-100k için başlangıç seviyesi normaldir).

  * P\@10 = %10 → ilk 10 öneride ort. 1 tanesi gerçekten pozitif.
  * R\@10 = %2.7 → kullanıcının tüm pozitiflerinden küçük bir kısmı yakalanıyor.
* Bu metrikleri yukarıdaki **tuning** adımlarıyla yavaş yavaş yükseltebilirsin.

---

hazırsan: istersen **ml-100k için film başlığı eşleştirme** (u.item formatı) örneğini de ayrıca atayım, ya da **Goodbooks-10k** gibi başka bir veri setine aynı RBM pipeline’ını uyarlayalım.
